# Импорт библиотек

In [80]:
import mlflow
from mlflow.tracking import MlflowClient

import pandas as pd
import numpy as np

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from transformers import pipeline
from transformers import Trainer, TrainingArguments

import joblib
import cloudpickle

import nltk
from nltk.tokenize import word_tokenize

nltk.download('stopwords')

from nltk.corpus import stopwords
from functools import lru_cache

from pymorphy3 import MorphAnalyzer
import re
import emoji

import warnings
warnings.filterwarnings('error')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Smart\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Эксперименты с датасетами

## Первый эксперимент (Лемматизация, удаление знаков пунктуации)

In [81]:
#!   mlflow server --host 127.0.0.1 --port 8080

In [82]:
# Подключаемся к серверу, запущенным на локальном хосте

mlflow.set_tracking_uri("http://127.0.0.1:8080")

In [83]:
analyzer = MorphAnalyzer(lang = 'ru')
stop_words = nltk.corpus.stopwords.words('russian')
stop_words_cleaned = []

for i in stop_words:
    if i != 'не' and i != 'нет' and i != 'никогда':
        stop_words_cleaned.append(i)


@lru_cache(maxsize=100_000)
def lemmatization(text):
    return analyzer.parse(text)[0].normal_form

def preprocess_text(text):

    text = re.sub(r'[\n\r\t]', ' ', text)
    
    text = re.sub(r'\bn(\w+)', r'\1', text)  # nчипсы -> чипсы
    
    text = re.sub(r'[^\w\s\d]', '', text)
    
    text = re.sub(r'\s+', ' ', text).strip()



    text_tokens = nltk.word_tokenize(text)
    sentence = []

    for i in text_tokens:
        if i not in stop_words_cleaned:
            sentence.append(lemmatization(i))

    return ' '.join(sentence)


print(preprocess_text('Всем привет! Какое же неприятное место, нет?\n'))



весь привет какой неприятный место нет


In [84]:
with mlflow.start_run(run_name='First dataset'):
    
    mlflow.set_tag('Dataset_version', '1.0.0')

    df_annotations_1 = pd.read_json("project-8-at-2025-11-01-11-43-483e368c.json")
    df_annotations_2 = pd.read_json("project-9-at-2025-11-01-11-51-0428f45f.json")
    df_annotations_3 = pd.read_json("project-10-at-2025-11-06-14-16-bb9bc9cd.json")

    df_annotations = pd.concat([df_annotations_1, df_annotations_2, df_annotations_3])

    df = pd.DataFrame(columns = ['span', 'label'])

    for mark in df_annotations['aspect_sentiment']:
        for param in range(len(mark)):
            span = mark[param]['text']
            label = mark[param]['labels'][0]

            df.loc[len(df)] = [span, label]

    df['span'] = df['span'].apply(preprocess_text)
    
    
    display(df)


    file = df.to_csv('First_version.csv')
    
    
    mlflow.log_artifact('First_version.csv', 'datasets')

    with open('preprocess_text_first.pkl', 'wb') as f: 
        preprocess_pickle = cloudpickle.dump(preprocess_text, f)

    mlflow.log_artifact('preprocess_text_first.pkl', 'functions')


,span,label
0,вкус реально отличный,ВКУС_POSITIVE
1,я есть чипсы очень долго год 2 назад советоват...,O
2,хотеться купить очень последний время стать оч...,O
3,вкус иметь каждый 2 пачка,O
4,вкус который стать намного острый обычный,ВКУС_NEGATIVE
...,...,...
2063,итогчипсы лейс flamin hot острый креветка васа...,ВКУС_POSITIVE
2064,у привлекательный аппетитный вид,ПАЧКА_POSITIVE
2065,оригинальный пикантный вкус,ВКУС_POSITIVE
2066,к покупка рекомендовать,ВКУС_POSITIVE


🏃 View run First dataset at: http://127.0.0.1:8080/#/experiments/0/runs/620c5c4ef684422da604d12fd6a457a1
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


## Второй эксперимент (Простая очистка от знаков препинания и изменение эмодзи)

In [85]:
def clean_text_only(text):

    if not isinstance(text, str):
        return ""
    
    text = text.lower()
    
    # Заменяем эмодзи
    text = emoji.demojize(text, delimiters=(" ", " "))
    
    # Удаляем переносы и спецсимволы
    text = re.sub(r'[\n\r\t]', ' ', text)
    
    # Удаляем пунктуацию
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Убираем лишние пробелы
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [86]:
with mlflow.start_run(run_name = 'Second dataset'):
    
    mlflow.set_tag('Dataset_version', '2.0.0')

    df_annotations_1 = pd.read_json("project-8-at-2025-11-01-11-43-483e368c.json")
    df_annotations_2 = pd.read_json("project-9-at-2025-11-01-11-51-0428f45f.json")
    df_annotations_3 = pd.read_json("project-10-at-2025-11-06-14-16-bb9bc9cd.json")

    df_annotations = pd.concat([df_annotations_1, df_annotations_2, df_annotations_3])

    df = pd.DataFrame(columns = ['span', 'label'])

    for mark in df_annotations['aspect_sentiment']:
        for param in range(len(mark)):
            span = mark[param]['text']
            label = mark[param]['labels'][0]

            df.loc[len(df)] = [span, label]
    
    df['span'] = df['span'].apply(clean_text_only)

    file = df.to_csv('Second_version.csv')
    
    display(df)

    mlflow.log_artifact('Second_version.csv', 'datasets')


    with open('preprocess_text_second.pkl', 'wb') as f:
        cloudpickle.dump(clean_text_only, f)

    mlflow.log_artifact('preprocess_text_second.pkl', 'functions')

,span,label
0,вкус был реально отличный,ВКУС_POSITIVE
1,я ел эти чипсы очень долго еще года 2 назад со...,O
2,хотелось купить очень много но в последнее вре...,O
3,сейчас такой вкус имеет каждая 2 пачка,O
4,вкус которых стал намного острее обычного,ВКУС_NEGATIVE
...,...,...
2063,check_mark_button итог nчипсы лейс flamin hot ...,ВКУС_POSITIVE
2064,у них привлекательный и аппетитный вид,ПАЧКА_POSITIVE
2065,оригинальный пикантный вкус,ВКУС_POSITIVE
2066,к покупке рекомендую,ВКУС_POSITIVE


🏃 View run Second dataset at: http://127.0.0.1:8080/#/experiments/0/runs/a9346d460ff042768bafb15e0fa0fb7d
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


## Третий эксперимент (Никакой очистки)

In [87]:
with mlflow.start_run(run_name = 'Third dataset'):
    
    mlflow.set_tag('Dataset_version', '3.0.0')

    df_annotations_1 = pd.read_json("project-8-at-2025-11-01-11-43-483e368c.json")
    df_annotations_2 = pd.read_json("project-9-at-2025-11-01-11-51-0428f45f.json")
    df_annotations_3 = pd.read_json("project-10-at-2025-11-06-14-16-bb9bc9cd.json")

    df_annotations = pd.concat([df_annotations_1, df_annotations_2, df_annotations_3])

    df = pd.DataFrame(columns = ['span', 'label'])

    for mark in df_annotations['aspect_sentiment']:
        for param in range(len(mark)):
            span = mark[param]['text']
            label = mark[param]['labels'][0]

            df.loc[len(df)] = [span, label]
    
    file = df.to_csv('Third_version.csv')
    
    display(df)

    mlflow.log_artifact('Third_version.csv', 'datasets')

,span,label
0,вкус был реально отличный,ВКУС_POSITIVE
1,"Я ел эти чипсы очень долго, еще года 2 назад с...",O
2,"хотелось купить очень много, но в последнее вр...",O
3,сейчас такой вкус имеет каждая 2 пачка,O
4,вкус которых стал НАМНОГО острее обычного,ВКУС_NEGATIVE
...,...,...
2063,"✅ ИТОГ\nЧипсы Лейс Flamin Hot ""Острая креветка...",ВКУС_POSITIVE
2064,У них привлекательный и аппетитный вид,ПАЧКА_POSITIVE
2065,оригинальный пикантный вкус.,ВКУС_POSITIVE
2066,К покупке рекомендую!,ВКУС_POSITIVE


🏃 View run Third dataset at: http://127.0.0.1:8080/#/experiments/0/runs/4f147edf8a714fd19e277cbe4bdf6f29
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


# Эксперементирование с моделями

## Первый эксперимент (Простая модель)

Будем обучать простую модель: "Наивный Байесовский Классификатор" из `Sklearn`, будем проводить тест на 3 версиях датасета и поймем, какая модель лучше себя покажет при работе с тем или иным датасетом

### Первый датасет

In [88]:
with mlflow.start_run(run_name = 'first_model_experiment'):

    mlflow.set_tag('NaiveBayes', '1.0.0')

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=['0'],  # Default эксперимент всегда имеет ID = 0
        filter_string="tags.mlflow.runName = 'First dataset'",
        order_by=['attributes.end_time desc']
    )

    first_dataset_latest_run = dataset_runs[0]
    
    first_dataset_latest_run_id = first_dataset_latest_run.info.run_id

    files = client.list_artifacts(first_dataset_latest_run_id, 'datasets')

    dataframe = files[0].path

    dataframe_path = client.download_artifacts(first_dataset_latest_run_id, dataframe)

    df = pd.read_csv(dataframe_path)

    display(df)



    # =====================================================================================================================================
    #                                              ВЕКТОРИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    vectoraizer = TfidfVectorizer(lowercase=True,
                                  analyzer = 'word',
                                  max_features=5000,
                                  ngram_range=(1, 4),
                                  min_df=2,
                                  max_df=0.9
                                  )
    
    encoder = LabelEncoder()



    # =====================================================================================================================================
    #                                         РАЗБИЕНИЕ НА TRAIN/TEST/VAL
    # =====================================================================================================================================


    df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

    X_train = df_train['span']
    X_train = vectoraizer.fit_transform(X_train)

    y_train = df_train['label']
    y_train = encoder.fit_transform(y_train)


    X_test = df_test['span']
    X_test = vectoraizer.transform(X_test)

    y_test = df_test['label']
    y_test = encoder.transform(y_test)


    # =====================================================================================================================================
    #                                         ОБУЧЕНИЕ И ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    mlflow.sklearn.autolog()

    model = MultinomialNB()

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    print(f'f1-score равен {f1_score(y_test, y_pred, average='weighted')}')
    print(f'accuracy-score равен {accuracy_score(y_test, y_pred)}')

    joblib.dump(model, 'naive_bayes.pkl')
    joblib.dump(vectoraizer, 'tfidfvectoraizer.pkl')
    joblib.dump(encoder, 'labelencoder.pkl')
    
    mlflow.log_artifact('tfidfvectoraizer.pkl', 'models')
    mlflow.log_artifact('labelencoder.pkl', 'models')
    mlflow.log_artifact('naive_bayes.pkl', 'models')

,Unnamed: 0,span,label
0,0,вкус реально отличный,ВКУС_POSITIVE
1,1,я есть чипсы очень долго год 2 назад советоват...,O
2,2,хотеться купить очень последний время стать оч...,O
3,3,вкус иметь каждый 2 пачка,O
4,4,вкус который стать намного острый обычный,ВКУС_NEGATIVE
...,...,...,...
2063,2063,итогчипсы лейс flamin hot острый креветка васа...,ВКУС_POSITIVE
2064,2064,у привлекательный аппетитный вид,ПАЧКА_POSITIVE
2065,2065,оригинальный пикантный вкус,ВКУС_POSITIVE
2066,2066,к покупка рекомендовать,ВКУС_POSITIVE


2025/11/13 14:01:46 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: Any type hint is inferred as AnyType, and MLflow doesn't validate the data for this type. Please use a more specific type hint to enable data validation.


f1-score равен 0.32907258187514044
accuracy-score равен 0.4033816425120773
🏃 View run first_model_experiment at: http://127.0.0.1:8080/#/experiments/0/runs/d1d9aaf965624a868fa3cfd9c14fafaa
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результатов

In [89]:
client = MlflowClient()


latest_run_model = client.search_runs(experiment_ids=['0'],
                         filter_string='attribute.run_name = "first_model_experiment"',
                         order_by=['attribute.end_time desc'])

latest_run_model_id = latest_run_model[0].info.run_id

print(latest_run_model_id)

model_run = client.get_run(latest_run_model_id)

artifacts = client.list_artifacts(latest_run_model_id, 'models')

# Получаем путь к нашим моделям из MLFlow

vectoraizer_file = client.download_artifacts(latest_run_model_id, 'models/tfidfvectoraizer.pkl')
bayes_file = client.download_artifacts(latest_run_model_id, 'models/naive_bayes.pkl')
encoder_file = client.download_artifacts(latest_run_model_id, 'models/labelencoder.pkl')


# Получаем путь к нашей функции предобработки данных из MLFlow


latest_run_dataset = client.search_runs(experiment_ids=['0'],
                                        filter_string='attributes.run_name="First dataset"',
                                        order_by=['attributes.end_time desc'])


latest_run_dataset_id = latest_run_dataset[0].info.run_id

art_loc = client.download_artifacts(latest_run_dataset_id, 'functions/preprocess_text_first.pkl')


# Ипортируем их в виде выполняемого скрипта

vectoraizer = joblib.load(vectoraizer_file)
bayes = joblib.load(bayes_file)
encoder = joblib.load(encoder_file)

with open(art_loc, 'rb') as f:
    preprocess_func = cloudpickle.load(f)


# Предсказываем метку текста

text = 'Меня привлекло такое сочетание'

preprocessed_text = preprocess_func(text)

done_text = vectoraizer.transform([preprocessed_text])
    
label = bayes.predict(done_text)

print('=' * 100)

print(f'Исходный текст: "{text}"')
print(f'Предобработанный текст: "{preprocessed_text}"')
print(encoder.inverse_transform(label))



d1d9aaf965624a868fa3cfd9c14fafaa


Исходный текст: "Меня привлекло такое сочетание"
Предобработанный текст: "я привлечь такой сочетание"
['ВКУС_POSITIVE']


### Второй датасет

In [90]:
with mlflow.start_run(run_name = 'second_model_experiment'):

    mlflow.set_tag('NaiveBayes', '2.0.0')

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=['0'],  # Default эксперимент всегда имеет ID = 0
        filter_string="tags.mlflow.runName = 'Second dataset'",
        order_by=['attributes.end_time desc']
    )

    first_dataset_latest_run = dataset_runs[0]
    
    first_dataset_latest_run_id = first_dataset_latest_run.info.run_id

    files = client.list_artifacts(first_dataset_latest_run_id, 'datasets')

    dataframe = files[0].path

    dataframe_path = client.download_artifacts(first_dataset_latest_run_id, dataframe)

    df = pd.read_csv(dataframe_path)

    display(df)



    # =====================================================================================================================================
    #                                              ВЕКТОРИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    vectoraizer = TfidfVectorizer(lowercase=True,
                                  analyzer = 'word',
                                  max_features=5000,
                                  ngram_range=(1, 4),
                                  min_df=2,
                                  max_df=0.9
                                  )
    
    encoder = LabelEncoder()



    # =====================================================================================================================================
    #                                         РАЗБИЕНИЕ НА TRAIN/TEST/VAL
    # =====================================================================================================================================


    df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

    X_train = df_train['span']
    X_train = vectoraizer.fit_transform(X_train)

    y_train = df_train['label']
    y_train = encoder.fit_transform(y_train)


    X_test = df_test['span']
    X_test = vectoraizer.transform(X_test)

    y_test = df_test['label']
    y_test = encoder.transform(y_test)


    # =====================================================================================================================================
    #                                         ОБУЧЕНИЕ И ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    mlflow.sklearn.autolog()

    model = MultinomialNB()

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    print(f'f1-score равен {f1_score(y_test, y_pred, average='weighted')}')
    print(f'accuracy-score равен {accuracy_score(y_test, y_pred)}')

    joblib.dump(model, 'naive_bayes.pkl')
    joblib.dump(vectoraizer, 'tfidfvectoraizer.pkl')
    joblib.dump(encoder, 'labelencoder.pkl')
    
    mlflow.log_artifact('tfidfvectoraizer.pkl', 'models')
    mlflow.log_artifact('labelencoder.pkl', 'models')
    mlflow.log_artifact('naive_bayes.pkl', 'models')

,Unnamed: 0,span,label
0,0,вкус был реально отличный,ВКУС_POSITIVE
1,1,я ел эти чипсы очень долго еще года 2 назад со...,O
2,2,хотелось купить очень много но в последнее вре...,O
3,3,сейчас такой вкус имеет каждая 2 пачка,O
4,4,вкус которых стал намного острее обычного,ВКУС_NEGATIVE
...,...,...,...
2063,2063,check_mark_button итог nчипсы лейс flamin hot ...,ВКУС_POSITIVE
2064,2064,у них привлекательный и аппетитный вид,ПАЧКА_POSITIVE
2065,2065,оригинальный пикантный вкус,ВКУС_POSITIVE
2066,2066,к покупке рекомендую,ВКУС_POSITIVE


2025/11/13 14:01:54 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: Any type hint is inferred as AnyType, and MLflow doesn't validate the data for this type. Please use a more specific type hint to enable data validation.


f1-score равен 0.31295994125933585
accuracy-score равен 0.37922705314009664
🏃 View run second_model_experiment at: http://127.0.0.1:8080/#/experiments/0/runs/f0238c9a62fd411ca1ed027b416391d4
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результатов

In [91]:
client = MlflowClient()


latest_run_model = client.search_runs(experiment_ids=['0'],
                         filter_string='attribute.run_name = "second_model_experiment"',
                         order_by=['attribute.end_time desc'])

latest_run_model_id = latest_run_model[0].info.run_id

print(latest_run_model_id)

model_run = client.get_run(latest_run_model_id)

artifacts = client.list_artifacts(latest_run_model_id, 'models')

# Получаем путь к нашим моделям из MLFlow

vectoraizer_file = client.download_artifacts(latest_run_model_id, 'models/tfidfvectoraizer.pkl')
bayes_file = client.download_artifacts(latest_run_model_id, 'models/naive_bayes.pkl')
encoder_file = client.download_artifacts(latest_run_model_id, 'models/labelencoder.pkl')


# Получаем путь к нашей функции предобработки данных из MLFlow


latest_run_dataset = client.search_runs(experiment_ids=['0'],
                                        filter_string='attributes.run_name="Second dataset"',
                                        order_by=['attributes.end_time desc'])


latest_run_dataset_id = latest_run_dataset[0].info.run_id

art_loc = client.download_artifacts(latest_run_dataset_id, 'functions/preprocess_text_second.pkl')


# Ипортируем их в виде выполняемого скрипта

vectoraizer = joblib.load(vectoraizer_file)
bayes = joblib.load(bayes_file)
encoder = joblib.load(encoder_file)

with open(art_loc, 'rb') as f:
    preprocess_func = cloudpickle.load(f)


# Предсказываем метку текста

text = 'Меня не привлекло такое сочетание 😭'

preprocessed_text = preprocess_func(text)

done_text = vectoraizer.transform([preprocessed_text])
    
label = bayes.predict(done_text)

print('=' * 100)

print(f'Исходный текст: "{text}"')
print(f'Предобработанный текст: "{preprocessed_text}"')
print(encoder.inverse_transform(label))

f0238c9a62fd411ca1ed027b416391d4


Исходный текст: "Меня не привлекло такое сочетание 😭"
Предобработанный текст: "меня не привлекло такое сочетание loudly_crying_face"
['ВКУС_POSITIVE']


### Третий датасет

In [92]:
with mlflow.start_run(run_name = 'third_model_experiment'):

    mlflow.set_tag('NaiveBayes', '3.0.0')

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=['0'],  # Default эксперимент всегда имеет ID = 0
        filter_string="tags.mlflow.runName = 'Third dataset'",
        order_by=['attributes.end_time desc']
    )

    first_dataset_latest_run = dataset_runs[0]
    
    first_dataset_latest_run_id = first_dataset_latest_run.info.run_id

    files = client.list_artifacts(first_dataset_latest_run_id, 'datasets')

    dataframe = files[0].path

    dataframe_path = client.download_artifacts(first_dataset_latest_run_id, dataframe)

    df = pd.read_csv(dataframe_path)

    display(df)



    # =====================================================================================================================================
    #                                              ВЕКТОРИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    vectoraizer = TfidfVectorizer(lowercase=True,
                                  analyzer = 'word',
                                  max_features=5000,
                                  ngram_range=(1, 4),
                                  min_df=2,
                                  max_df=0.9
                                  )
    
    encoder = LabelEncoder()



    # =====================================================================================================================================
    #                                         РАЗБИЕНИЕ НА TRAIN/TEST/VAL
    # =====================================================================================================================================


    df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

    X_train = df_train['span']
    X_train = vectoraizer.fit_transform(X_train)

    y_train = df_train['label']
    y_train = encoder.fit_transform(y_train)


    X_test = df_test['span']
    X_test = vectoraizer.transform(X_test)

    y_test = df_test['label']
    y_test = encoder.transform(y_test)


    # =====================================================================================================================================
    #                                         ОБУЧЕНИЕ И ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    mlflow.sklearn.autolog()

    model = MultinomialNB()

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    print(f'f1-score равен {f1_score(y_test, y_pred, average='weighted')}')
    print(f'accuracy-score равен {accuracy_score(y_test, y_pred)}')

    joblib.dump(model, 'naive_bayes.pkl')
    joblib.dump(vectoraizer, 'tfidfvectoraizer.pkl')
    joblib.dump(encoder, 'labelencoder.pkl')
    
    mlflow.log_artifact('tfidfvectoraizer.pkl', 'models')
    mlflow.log_artifact('labelencoder.pkl', 'models')
    mlflow.log_artifact('naive_bayes.pkl', 'models')

,Unnamed: 0,span,label
0,0,вкус был реально отличный,ВКУС_POSITIVE
1,1,"Я ел эти чипсы очень долго, еще года 2 назад с...",O
2,2,"хотелось купить очень много, но в последнее вр...",O
3,3,сейчас такой вкус имеет каждая 2 пачка,O
4,4,вкус которых стал НАМНОГО острее обычного,ВКУС_NEGATIVE
...,...,...,...
2063,2063,"✅ ИТОГ\nЧипсы Лейс Flamin Hot ""Острая креветка...",ВКУС_POSITIVE
2064,2064,У них привлекательный и аппетитный вид,ПАЧКА_POSITIVE
2065,2065,оригинальный пикантный вкус.,ВКУС_POSITIVE
2066,2066,К покупке рекомендую!,ВКУС_POSITIVE


2025/11/13 14:02:04 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: Any type hint is inferred as AnyType, and MLflow doesn't validate the data for this type. Please use a more specific type hint to enable data validation.


f1-score равен 0.31762069535565496
accuracy-score равен 0.38164251207729466
🏃 View run third_model_experiment at: http://127.0.0.1:8080/#/experiments/0/runs/ced8f567612044cea4117d879408d2fe
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результатоы

In [93]:
client = MlflowClient()


latest_run_model = client.search_runs(experiment_ids=['0'],
                         filter_string='attribute.run_name = "third_model_experiment"',
                         order_by=['attribute.end_time desc'])

latest_run_model_id = latest_run_model[0].info.run_id

print(latest_run_model_id)

model_run = client.get_run(latest_run_model_id)

artifacts = client.list_artifacts(latest_run_model_id, 'models')

# Получаем путь к нашим моделям из MLFlow

vectoraizer_file = client.download_artifacts(latest_run_model_id, 'models/tfidfvectoraizer.pkl')
bayes_file = client.download_artifacts(latest_run_model_id, 'models/naive_bayes.pkl')
encoder_file = client.download_artifacts(latest_run_model_id, 'models/labelencoder.pkl')


# Ипортируем их в виде выполняемого скрипта

vectoraizer = joblib.load(vectoraizer_file)
bayes = joblib.load(bayes_file)
encoder = joblib.load(encoder_file)

with open(art_loc, 'rb') as f:
    preprocess_func = cloudpickle.load(f)


# Предсказываем метку текста

text = 'Меня не привлекло такое сочетание 😭'

preprocessed_text = preprocess_func(text)

done_text = vectoraizer.transform([preprocessed_text])
    
label = bayes.predict(done_text)

print('=' * 100)

print(f'Исходный текст: "{text}"')
print(f'Предобработанный текст: "{preprocessed_text}"')
print(encoder.inverse_transform(label))

ced8f567612044cea4117d879408d2fe


Исходный текст: "Меня не привлекло такое сочетание 😭"
Предобработанный текст: "меня не привлекло такое сочетание loudly_crying_face"
['ВКУС_POSITIVE']


### Итог

In [94]:
results = pd.DataFrame({
    'Версия': ['Версия 1', 'Версия 2', 'Версия 3'],
    'F1-Score': [0.363707, 0.344306, 0.344139],
    'Accuracy': [0.413043, 0.391304, 0.391304],
    'Описание': ['Лемматизация, удаление знаков пунктуации', 'Очистка от знаков препинания и emoji to text', 'Никакой очистки']
})

results

,Версия,F1-Score,Accuracy,Описание
0,Версия 1,0.363707,0.413043,"Лемматизация, удаление знаков пунктуации"
1,Версия 2,0.344306,0.391304,Очистка от знаков препинания и emoji to text
2,Версия 3,0.344139,0.391304,Никакой очистки
